# Занятие 13
# Прикладная алгебра и численные методы
## Характеристический многочлен и собственные числа матриц. QR-разложение.

In [2]:
import numpy as np
import sympy
import scipy.linalg
import numpy.linalg
from numpy import random
from sympy import Matrix, S, Symbol, symbols, I, zeros, eye
from sympy import simplify, expand, expand_complex, latex
import pandas as pd
from google.colab import files
from IPython.display import Latex

## Характеристический многочлен.
$$
\chi_A(\lambda) = \det(A - \lambda E),
$$
$\lambda_1$, $\lambda_2$, ..., $\lambda_n$ - корни характеристического многочлена, собственные числа  матрицы $A$.

Разложение комплексного многочлена от одной переменной на линейные множители:
$$
f(x) = a_0 + a_1 x + ... + a_{n - 1} x^{n - 1} + x^n = (x - x_1)...(x - x_n).
$$
Формулы Виета:
\begin{align*}
&a_{n - 1} = -(x_1 + x_2 + ... + x_n)\\
&a_{n - 2} = (x_1x_2 + x_1x_3 + ... + x_{n - 1}x_n)\\
&a_{n - 3} = -(x_1x_2x_3 + x_1x_2x_4 + ... + x_{n - 2}x_{n - 1}x_n)\\
&...\\
&a_{n - k} = (-1)^k\sum_{1\le i_1<..<i_k\le n}x_{i_1} ... x_{i_k}\\
&...\\
&a_0 = (-1)^n x_1 ... x_n.
\end{align*}
Элементарный симметрический многочлен от $x_1$, ... $x_n$:
$$
s_k = s_k(x_1 ... x_n) = \sum_{1\le i_1<..<i_k\le n}x_{i_1} ... x_{i_k}, \quad k = 1,...,n.
$$
Выражение для коэффициентов многочлена $f(x)$:
$$
a_{n - k} = (-1)^ks_k, \quad k = 1,...,n,
$$
где $x_1$, ... $x_n$ - корни $f(x)$.
## Пример 1.
Составим многочлен, корнями которого являются числа $2 - i$, $-1 + 2i$, $-2$, $5$.
Вычислим коэффициенты многочлена по корням.

Составим выражение в sympy, затем на основе этого выражения получим функцию, которую можно использовать в numpy.


In [3]:
from sympy.abc import x

roots = [2 - sympy.I, -1 + 2 * sympy.I, -2, 5]
roots_np = np.array(roots, dtype=complex)
coeffs = [-sum(roots),
           sum([roots[i] * roots[j] for i in range(4) for j in range(i + 1, 4)]),
          -sum([roots[i] * roots[j]*roots[k] for i in range(4) for j in range(i + 1, 4) for k in range(j + 1, 4)]),
         (-1) ** 4 * roots[0] * roots[1] * roots[2] * roots[3]]
coeffs = [1] + [item.simplify() for item in coeffs]        
my_poly = sum([coeffs[k] * x ** (4 - k) for k in range(5)])   
display(my_poly)

numpy_poly = sympy.lambdify(x, my_poly, 'numpy')
display(*[Latex(f"""f(x_{k}) = {my_poly.subs(x, root).simplify()} =
 {numpy_poly(root_np)}""") for k, root, root_np in zip(range(len(roots)),
                                                       roots, roots_np)])

x**4 + x**3*(-4 - I) + x**2*(-7 + 8*I) + x*(10 - 5*I) - 50*I

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

То же самое, но средствами sympy:

In [4]:
display(sum([(-1) ** k * x ** (4 - k) * (sympy.polys.specialpolys.symmetric_poly(k, roots).simplify().expand()) for k in range(5)]))

x**4 - x**3*(4 + I) + x**2*(-7 + 8*I) - x*(-10 + 5*I) - 50*I

 Теперь numpy:

In [5]:
display(Latex(latex(np.polynomial.polynomial.polyfromroots(roots_np))))

<IPython.core.display.Latex object>

## Формулы Ньютона
Пусть задан многочлен $f(\lambda) = (-1)^n( \lambda^n + b_1 \lambda^{n - 1}  + b_2 \lambda^{n - 2} + ... +  b_{n - 1} \lambda+ b_n)$, причем $\lambda_1$, ... $\lambda_n$ - все корни $f(\lambda)$ с учетом кратностей.
### Ньютоновы суммы для $f(\lambda)$ - выражения
$$
p_k = \lambda_1^k + \lambda_2^k +...+\lambda_n^k,\quad k = 1, 2, ....
$$
### Формулы Ньютона
\begin{align*}
&p_1 = s_1\\
&p_2 = s_1^2 - 2s_2\\
&....\\
&k s_k = \sum_{i = 1}^k (-1)^{i - 1} s_{k - i} p_i\\
&b_k = \frac{-1}{k}(p_k + b_1 p_{k - 1} + ... + b_{k - 1} p_1)
\end{align*}
Значения $p_k$ находятся так:  $p_k = tr A^k$.


Если $|x_1| > |x_2| \ge ... \ge |x_n|$, то
$$
\frac{s_{k + 1}}{s_k}\to x_{\max},
$$
где $x_{\max}$ - наибольший по модулю корень $f(x)$ кратности 1.
## Пример 2.
Для корней Примера 1 вычислим коэффициенты многочлена по методу Ньютона.

In [6]:
p_k = [sum([item ** k for item in roots]).simplify() for k in range(1, 5)]
b1 = -p_k[0]
b2 = -((p_k[1] + b1 * p_k[0]) / 2).simplify()
b3 = -((p_k[2] + b1 * p_k[1] + b2 * p_k[0]) / 3).simplify()
b4 = -((p_k[3] + b1 * p_k[2] + b2 * p_k[1] + b3 * p_k[0]) / 4).simplify()
my_poly2 = sum([item * x ** (4 - k) for k, item in enumerate([1, b1, b2, b3, b4])])   
display(my_poly, my_poly2, *p_k)


x**4 + x**3*(-4 - I) + x**2*(-7 + 8*I) + x*(10 - 5*I) - 50*I

x**4 + x**3*(-4 - I) + x**2*(-7 + 8*I) + x*(10 - 5*I) - 50*I

4 + I

29 - 8*I

130 - 13*I

627

## Пример 3.
По формулам Ньютона найдем характеристический многочлен матрицы из файла Problem_3_13.xlsx.

In [4]:
uploaded = files.upload()
for fn3 in uploaded.keys():
  print(f'User uploaded file "{fn3}" with length {len(uploaded[fn3])} bytes')

Saving Problem3_13.xlsx to Problem3_13.xlsx
User uploaded file "Problem3_13.xlsx" with length 7961 bytes


In [5]:
n = pd.read_excel(fn3, index_col=None, header=None).to_numpy().shape[1]
A3 = pd.read_excel(fn3, index_col=None, header=None, 
                   converters={i: complex for i in range(n)}).to_numpy()
p_k = [np.trace(np.linalg.matrix_power(A3, k)) for k in range(1, n + 1)]
b1 = -p_k[0]
b2 = -((p_k[1] + b1 * p_k[0]) / 2)
b3 = -((p_k[2] + b1 * p_k[1] + b2 * p_k[0])/3)
b4 = -((p_k[3] + b1 * p_k[2] + b2 * p_k[1] + b3 * p_k[0]) / 4)
my_poly3 = sum([item * x ** (4 - k) for k, item in enumerate([1, b1, b2,
                                                              b3, b4])])   
display(my_poly3)

x**4 - 24.0*x**3 + 75.0*x**2 + 222.0*x - 20.0

## Собственные значения неотрицательной неразложимой матрицы.
Собственное значение $\lambda$ матрицы $A$ - такое число, что существует ненулевой вектор $x$, для которого $Ax = \lambda x$, что эквивалентно $(A - \lambda E) x = 0$ ($0$ - нулевой вектор!).

Если матрица  $A \ge 0$ неразложима, то по теореме Фробениуса-Перрона
$$
\hat\lambda = |\lambda_{\max}| = \rho(A),
$$
В этом случае для любого начального приближения $x^0$ последовательность
$$
x^{n + 1} = \frac{1}{|Ax^{n}|_1}Ax^n \qquad(1)
$$
сходится к собственному вектору $\hat x$, соответствующему собственному значению $\hat\lambda$.

Если при этом $\hat x_i > 0$, то $\hat\lambda \approx \frac{(x^{n + 1})_i}{(x^n)_i}$.

## Пример 4.
Составим матрицу из положительных чисел от 1 до 10 размерности 10 и будем вычислять элементы итерационной последовательности по формуле (1), пока последовательные векторы $x^k$ и $x^{k + 1}$ не будут отличаться менее чем на $\varepsilon = 0.001$ по норме 1.


In [9]:
n = 10
eps = 0.001
A1 = np.array([[random.randint(1, 10) for _ in range(n)] for _ in range(n)])
x1 = A1[:, 0]
x0 = x1 + np.ones((n, 1))
while np.linalg.norm(x1 - x0, ord=1) >= eps:
  x0 = x1
  Ax0 = A1 @ x0
  x1 = Ax0 / np.linalg.norm(Ax0, ord=1)
  print(f'x = {x1.T.round(3)}')
lambdaA1 = (A1 @ x1)[0] / x1[0]
print(f'с.в. {x1.T.round(3)}\nс.з. {lambdaA1.round(3)}\nAx = lambda x {np.allclose(A1 @ x1, lambdaA1 * x1, atol=eps, rtol=eps)}')   

x = [0.112 0.083 0.08  0.086 0.101 0.114 0.099 0.123 0.096 0.107]
x = [0.092 0.084 0.094 0.076 0.109 0.124 0.091 0.119 0.105 0.106]
x = [0.088 0.084 0.097 0.074 0.112 0.124 0.09  0.119 0.106 0.106]
x = [0.088 0.084 0.097 0.074 0.112 0.124 0.09  0.119 0.106 0.106]
x = [0.088 0.084 0.097 0.074 0.112 0.124 0.09  0.119 0.106 0.106]
с.в. [0.088 0.084 0.097 0.074 0.112 0.124 0.09  0.119 0.106 0.106]
с.з. 47.208
Ax = lambda x True


Вариант без риска зацикливания:

In [10]:
max_iter_num = 10**5
x1 = A1[:, 0]
x0 = x1 + np.ones((n, 1))
for i in range(max_iter_num):
  x0 = x1
  Ax0 = A1 @ x0
  x1 = Ax0 / np.linalg.norm(Ax0, ord=1)
  if np.linalg.norm(x1 - x0, ord=1) < eps:
    break
  
print(f'итерация {i}: x{i} = {x1.T.round(3)}')
lambdaA1 = (A1 @ x1)[0] / x1[0]
print(f"""с.в. {x1.T.round(3)}\nс.з. {lambdaA1.round(3)}\n
Ax = lambda x {np.allclose(A1 @ x1, lambdaA1 * x1, atol=eps, rtol=eps)}""")    

итерация 4: x4 = [0.088 0.084 0.097 0.074 0.112 0.124 0.09  0.119 0.106 0.106]
с.в. [0.088 0.084 0.097 0.074 0.112 0.124 0.09  0.119 0.106 0.106]
с.з. 47.208

Ax = lambda x True


Проверим, что получилось собственное значение, равное спектральному радиусу. Также проверим, что собственный вектор, соответствующий спектральному радиусу, имеет координаты, пропорциональные координатам полученного нами вектора.

In [11]:
spectrA1, eigvectorsA1 = np.linalg.eig(A1)
spectrA1 = np.round(spectrA1)
max_ev = eigvectorsA1[:, 0]
print(f"""спектральный радиус {abs(spectrA1[0])}
соответствующий собственный вектор {max_ev.T.real.round(3)}
пропорцинален вектору {x1.T.round(3)}? 
{np.allclose(x1 / x1[0], max_ev / max_ev[0], atol=10 ** (-3))}""")

спектральный радиус 47.0
соответствующий собственный вектор [-0.274 -0.262 -0.305 -0.231 -0.35  -0.389 -0.28  -0.373 -0.331 -0.33 ]
пропорцинален вектору [0.088 0.084 0.097 0.074 0.112 0.124 0.09  0.119 0.106 0.106]? 
True


Можно сравнить собственные векторы и иначе - нормировав кажый из них по одной и той же норме, например, 1.

!!! Важно: все координаты max_ev отрицательные, поэтому для корректного сравнения умножим $x_1$ на $-1$.

In [12]:
np.allclose(*[item / np.linalg.norm(item, ord=1) for item in (-x1, max_ev)], atol=10**(-2))

True

## Круги Гершгорина
#### Первая теорема Гершгорина

Все собственные значения матрицы $A_{n\times n} = (a_{ij})$ содержатся в объединении кругов на комплексной плоскости
$$
|\lambda - a_{ii}| \le \sum_{k = 1, k \neq i}^n |a_{ik}|, \quad i = 1, ..., n.
$$

## Вторая теорема Гершгорина

Если объединение $U$ $r$ кругов Гершгорина не пересекается с остальными $n - r$ кругами, то $U$ содержит ровно $r$ собственных значений матрицы $A_{n\times n} = (a_{ij})$ с учетом кратности.


### QR разложение
$A = QR$ для симметричной вещественной матрицы $A$

$Q$ - матрица из ортогональных столбцов, т.е. $Q^HQ = I$, $I$ - единичная матрица, причем может не выполняться $QQ^H = I$ (для ортогональной матрицы $Q^HQ = QQ^H = I$),
$R$ - правая треугольная (трапециевидная) матрица.

Ранг матрицы $A$ равен числу столбцов матрицы $Q$.
### Пример 5.
Построим  QR разложение для матрицы 
$$
M=\left(
\begin{matrix}
0&3+ I&5 - 2I&1\\
 - I&2&1 - I&2\\
5&-1 + 4I&-3&3
\end{matrix}
\right)
$$

In [14]:
M = Matrix([[0, 3 + I, 5 - 2 * I, 1], [-I, 2, 1 - I, 2], [5, -1 + 4 * I, -3, 3]])
Q, R = M.QRdecomposition()
MQR = Q * R
Q, R, MQR = [simplify(expand(item)) for item in (Q, R, MQR)]
display(Latex("""M = {}\\\\Q = {}, R = {}\\\\
QR = {}\\\\M = QR\ {}""".format(*map(latex, (M, Q, R, MQR, M == MQR))))) 

<IPython.core.display.Latex object>

## Решение систем линейных уравнений с помощью разложений.
### Пример 6.
Решим с помощью  QR разложения матрицы
$$
B=\left(
\begin{matrix}
1 &  2 & 3\\
4 & 5 &  6\\
7 &  8 & 9
\end{matrix}
\right)
$$
систему линейных уравнений
$$
BX = b,
\quad
b = \left(
\begin{matrix}6\\12\\24\end{matrix}
\right).
$$
Проверим совместность СЛАУ:

In [15]:
B = Matrix([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
b = Matrix([6, 12, 24])
Bb = B.row_join(b)
print(f"""B.rank = {B.rank()}, Bb.rank = {Bb.rank()},
B.rank == Bb.rank(): {B.rank() == Bb.rank()}""")

B.rank = 2, Bb.rank = 3,
B.rank == Bb.rank(): False


СЛАУ несовместна, в обычном смысле решения нет, но с помощью QR разложения можно найти псевдорешение, т.е. такую матрицу- столбец, что при подстановке в СЛАУ вместо $X$ даст минимальную возможную норму разности левой и правой частей СЛАУ (невязки).

In [16]:
X = B.QRsolve(b)
display(X)

Matrix([
[1],
[2]])

Для сокращения объема используемой памяти используется сокращенная форма QR разложения, при которой прямоугольная часть матрицы Q, состоящая из одних нулей, не хранится. Для восстановления полного решения в нашем случае достаточно добавить один ноль к получившемуся столбцу (поскольку столбце X должен состоять из трех элементов):

In [17]:
X = X.col_join(Matrix([0]))
delta = B * X - b
display(Latex('X = {}, delta = {}, delta.norm(2) = {}'.format(*map(latex, (X, delta, delta.norm(2))))))

<IPython.core.display.Latex object>

Полученное псевдорешение не является нормальным псевдорешением, т.е. псевдорешением с минимальной нормой, но и нормальное псевдорешение проще получить, используя QR разложение.

## Жорданова форма матрицы
$A = PJP^{-1}$ для квадратной матрицы $A$

$P$ - матрица перехода, $J$ - жорданова матрица.

### Пример 7.
Построим  жорданову форму для матриц
$$
B=\left(
\begin{matrix}
6 &  5 & -2\\
-3 & -1 &  3\\
2 &  1 & -2
\end{matrix}
\right)
\quad
K=\left(
\begin{matrix}
6 &  5 & -2 & -3\\
-3 & -1 &  3 &  3\\
2 &  1 & -2 &  -3\\
-1 & 1 & 5 & 5
\end{matrix}
\right)
$$

In [18]:
B = Matrix([[6,  5, -2], [-3, -1,  3], [2,  1, -2]])
P, J = B.jordan_form()
P, J = [simplify(expand(item)) for item in (P, J)]
display(Latex('P = {}, J = {}\\\\PJP^{{-1}} = {}, B  = {}'.format(*map(latex, (P, J, P * J * P ** (-1), B)))))
K = Matrix([[6,  5, -2, -3], [-3, -1,  3,  3], [2,  1, -2, -3], [-1,  1,  5,  5]])
P, J = K.jordan_form()
P, J = [simplify(expand(item)) for item in (P, J)]
display(Latex('P = {}, J = {}\\\\PJP^{{-1}} = {}, K  = {}'.format(*map(latex, (P, J, P * J * P ** (-1), K)))))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>